Load Twitter Data

In [9]:
import gensim
import pandas as pd
import pyodbc
import re
from gensim.models import Word2Vec


In [2]:
import configLocal
server = configLocal.server
database = configLocal.database
username = configLocal.username
password = configLocal.password
cnxn = configLocal.cnxn
cursor = cnxn.cursor()

data = pd.read_sql_query("select msg_id, msg_users_id, msg_timestamp, msg_text from dbo.msg", cnxn)

In [7]:
#parse database to dict
corpus = {}
for i in range(data.shape[0]):
    if data.iloc[i]["msg_users_id"] in corpus:
        corpus[data.iloc[i]["msg_users_id"]].append(re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE))
    else:
        corpus[data.iloc[i]["msg_users_id"]] = [re.sub(r'http\S+', '', data.iloc[i]["msg_text"], flags=re.MULTILINE)]

print(type(corpus), len(corpus))

for i in corpus.keys():
    corpus[i] = gensim.utils.simple_preprocess("".join(corpus[i]).encode('utf-8'), deacc=True, min_len=3)

<class 'dict'> 5624


carga del giga modelo

In [12]:
#now, lets load da big boi model

from numpy import  zeros, float32 as REAL
from gensim.models import keyedvectors
import codecs

# this function was build using code excerpts from:
# https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/keyedvectors.py
def load_vectors_from_csv(fname,vocab_size=973265,vector_size=100):
    print("Loading vectors from file:",fname)
    result=keyedvectors.KeyedVectors(vector_size=100)
    result.syn0 = zeros((vocab_size, vector_size), dtype=REAL)
    result.vector_size=vector_size
    counts=None   
    def add_word(word, weights):
        word_id = len(result.vocab)
        if word in result.vocab:
            print("duplicate word '%s' in %s, ignoring all but first", word, fname)
            return
        if counts is None:
            # most common scenario: no vocab file given. just make up some bogus counts, in descending order
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=vocab_size - word_id)
        elif word in counts:
            # use count from the vocab file
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=counts[word])
        else:
            # vocab file given, but word is missing -- set count to None (TODO: or raise?)
            print("vocabulary file is incomplete: '%s' is missing", word)
            result.vocab[word] = keyedvectors.Vocab(index=word_id, count=None)
        result.syn0[word_id] = weights
        result.index2word.append(word)   
    file=codecs.open(fname,"r","utf-8")
    i=0
    for line in file:
        i+=1
        if i==1: #ommit header
            continue
        parts=line.strip().split(",")
        word,weights=parts[1],[REAL(x) for x in parts[2:]]
        add_word(word,weights)
        if i%100000==0:
            print(i,"word vectors loaded so far ...")
    file.close()
    print(i-1,"word vectors loaded!")
    return result
    

    
model=load_vectors_from_csv("data/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos2.csv")

Loading vectors from file: data/WORD2VEC-Twitter-Espa_ol_para_Latinoam_rica__Espa_a_y_Estados_Unidos2.csv


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).


100000 word vectors loaded so far ...
159742 word vectors loaded!


--------------

In [170]:
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.manifold import TSNE
from collections import Counter
from six.moves import cPickle
import gensim.models.word2vec as w2v
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import multiprocessing
import os
import sys
import io
import re
import json
from tqdm import tqdm_notebook as tqdm
from string import punctuation
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("spanish")

In [136]:
#generar una lista con los tweets sucios
df = [x for x in data.msg_text]

In [144]:
#funcion de limpieza:
def process_raw_data(input_file):
    url_match = "(https?:\/\/[0-9a-zA-Z\-\_]+\.[\-\_0-9a-zA-Z]+\.?[0-9a-zA-Z\-\_]*\/?.*)"
    name_match = "\@[\_0-9a-zA-Z]+\:?"
        
    lines = input_file
    num_lines = len(input_file)
    ret = []
    for text in tqdm(lines):
        text = re.sub(url_match, u"", text)
        text = re.sub(name_match, u"", text)
        text = re.sub("\&amp\;?", u"", text)
        text = re.sub("[\:\.]{1,}$", u"", text)
        text = re.sub("^RT\:?", u"", text)
        text = u''.join(text)
        text = text.strip()
        ret.append(text)
    return ret

In [145]:
df_proc = process_raw_data(df)

In [180]:
#toquenizado
def tokenize_sentences(sentences):
    ret = []
    max_s = len(sentences)
    print("Got " + str(max_s) + " sentences.")
    for count, s in enumerate(sentences):
        tokens = []
        words = re.split(r'(\s+)', s)
        if len(words) > 0:
            for w in words:
                if w is not None:
                    w = w.strip()
                    w = w.lower()
                    w = ''.join(c for c in w if c not in punctuation)
                    if w.isspace() or w == "\n" or w == "\r":
                        w = None
                    if len(w) < 1:
                        w = None
                    if w is not None:
                        tokens.append(w)
        if len(tokens) > 0:
            ret.append(tokens)
    return ret

In [184]:
df_proc_T = tokenize_sentences(df_proc)

Got 30780 sentences.


In [201]:
#limpieza de los tokens
def clean_sentences(tokens):
    ret = []
    max_s = len(tokens)
    for sentence in tqdm(tokens):
        cleaned = []
        for token in sentence:
            if len(token) > 0:
                if cachedStopWords is not None:
                    for s in cachedStopWords:
                        if token == s:
                            token = None
                if token is not None:
                    if re.search("^[0-9\.\-\s\/]+$", token):
                        token = None
                if token is not None:
                    cleaned.append(token)
        if len(cleaned) > 0:
            ret.append(cleaned)
    return ret

In [204]:
df_proc_T_C = clean_sentences(df_proc_T)
df_proc_T_C

[['jo', 'també', 'però', 'diria', 'reflexiu', 'és', 'sobrer'],
 ['fuerte', 'lastima'],
 ['bro',
  'quejo',
  'si',
  'parece',
  'puta',
  'madre',
  'mejores',
  'casters',
  'lvp',
  'solo',
  'comentaba'],
 ['evidente'],
 ['karen', 'manda', 'msj', 'días'],
 ['autónomos', 'alertan', 'creación', 'empleo', 'desplomará', 'año'],
 ['madurao', 'jajaja'],
 ['quiero'],
 ['i’m',
  'currently',
  'doing',
  'the',
  'same',
  'get',
  'you',
  'perfectly',
  'ahaha'],
 ['engañó', 'principio'],
 ['llorente', 'pointe'],
 ['city', 'bar', 'gastronomía', 'vía', 'laietana'],
 ['ningún',
  'momento',
  'puse',
  'ver',
  'simplemente',
  'sorprendió',
  'siempre',
  'critiques',
  'compartas',
  'rato',
  'portadas',
  'propagandística',
  'dijeras',
  'verdá',
  'produzca',
  'menos',
  'grave',
  'emita'],
 ['siento',
  'solisima',
  'juro',
  'veo',
  'contarle',
  'puedo',
  'desahogarme',
  'creo',
  'q',
  'necesito'],
 ['si', 'habitacion', 'techo', 'mojaria', 'si', 'lloviera'],
 ['pues',
  's

https://labsblog.f-secure.com/2018/01/30/nlp-analysis-of-tweets-using-word2vec-and-t-sne/


The next function creates a vocabulary from the processed text. A vocabulary, in this context, is basically a list of all unique tokens in the data. This function creates a frequency distribution of all tokens (words) by counting the number of occurrences of each token. We will use this later to “trim” the vocabulary down to a manageable size.